# Post-analysis

This is how you can analyze your experiment: statistical tests, graphics and reports.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import sys
import logging
import pandas as pd
import numpy as np
import yaml

sys.path.append('../')

from abacus.auto_ab.abtest import ABTest
from abacus.auto_ab.params import ABTestParams
from abacus.auto_ab.params import DataParams, HypothesisParams

logging.basicConfig(level = logging.INFO)


%load_ext autoreload
%autoreload 2

## Case: continuous metric

In [ ]:
df = pd.read_csv('./data/ab_data.csv')

In [ ]:
data_params = DataParams(
    id_col='id', 
    group_col='groups', 
    strata_col='country', 
    target='height_now', 
    covariate='height_prev'
)

hypothesis_params = HypothesisParams(
    alpha=0.05, 
    beta=0.2, 
    alternative='two-sided',
    metric_type='continuous', 
    metric_name='median', 
    n_buckets=1000
)

ab_params = ABTestParams(data_params, hypothesis_params)

In [ ]:
ab_test = ABTest(df, ab_params)
ab_test_1 = ab_test.cuped()
ab_test_2 = ab_test_1.bucketing()

In [ ]:
print(f"bootstrap_test: {ab_test_2.test_boot_confint()}")
print(f"mannwhitney_test: {ab_test_2.test_mannwhitney()}")
print(f"welch_test: {ab_test_2.test_welch()}")

In [ ]:
ab_test_2.report()

In [ ]:
ab_test_2.plot()

## Case: binary metric

In [ ]:
df = pd.read_csv('./data/ab_data.csv')

In [ ]:
data_params = DataParams(
    id_col='id', 
    group_col='groups', 
    target_flg='conversion'
)


hypothesis_params = HypothesisParams(
    alpha=0.05, 
    beta=0.2, 
    alternative='two-sided', 
    metric_type='binary', 
    metric_name='mean', 
)

ab_params = ABTestParams(data_params,hypothesis_params)

In [ ]:
ab_test = ABTest(df, ab_params)

In [ ]:
print(f"ztest_test: {ab_test.test_z_proportions()}")

In [ ]:
ab_test.report()

In [ ]:
ab_test.plot()